<a href="https://colab.research.google.com/github/dream80/DFLWorkspace/blob/master/DeepFaceLab_Colab_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

无需多说，一路运行即可！

此脚本由**[托尼是塔克](https://wxp123.me)**创建，仅用于学习，请勿滥用。如有疑问，拉到最后！

In [0]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

Wed Nov 13 12:36:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 第一步 准备好workspace
这一步你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。



谷歌云盘地址：https://drive.google.com/drive/my-drive  
注意：谷歌网址现在都需要“科学上网”才能访问。  

In [0]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


挂载成功后，可以加载一个官方的workspace 也可以通过谷歌云盘形式上传自己的workspace.到指定路径  
注意上传的压缩包用zip格式，文件名必须是workspace.zip   
建议这个workspace只包含src,dst等内容，不包含model，这样第六就不会覆盖model文件了。  
model文件比较少可以不压缩直接上传到model目录。

In [0]:
#@title 加载示例项目workspace 
# 1. 练手可以使用这一行，直接git clone一个workspace
%cd /content/drive/My Drive/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

In [0]:
#@title 解压自己的项目workspace
#需要先将workspace.zip 上传到谷歌云盘的DeepFaceLab目录下面
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  workspace.zip

#第二步 安装DeepFaceLab
获取源代码，安装依赖,根据自己的情况选择版本  
620：稳定版，比较好的一个版本  
1019:支持SAEHD的版本  
last:最新版  

In [0]:
#@title 开始安装

Version = "v1.11.09" #@param ["620", "v1.10.19","v1.11.09","v1.11.12", "last"]
!rm -fr DeepFaceLab_Colab
if Version=="620":
  print("620版加载中....")
  # 获取DFL源代码v1.6.1稳定版
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements_colab.txt  
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt
else:
  print(Version+"加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


# 第三步. 提取脸部

这个步骤其实是分两次的。第一次转src，第二次选dst，很多对Deepfacelab不了解的人可能只点了一个，后面就会报错了。

In [0]:
#@title 开始提取
target = "dst" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug
  


In [0]:
#@title 开始排序
target = "src" #@param ["src","dst"]
order = "hist" #@param ["blur", "face", "face-dissim", "face-yaw", "face-pitch", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-no-blur"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned --by "+order
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned --by "+order

!python $cmd

# 第四步. 训练模型


*  支持H128,SAE,DF, LIAEF128等模型，根据自己的情况选择模型。
*  第一次需要配置模型参数，不懂的直接回车默认。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [0]:
#@title 开始训练
Model = "Quick96" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR","Quick96"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

**模型预览方法请参考：**
https://www.wxp123.me/p/329

简单描述：左侧->文件->/driver/..../model/ *preview*.jpg->双击  
  
这种方法比在脚本中预览和在网盘预览都要方便，并且无需启动预览配置！

# 第五步. 转换输出

训练模型的时候选了什么，这里就选什么  
比如训练H128,这里就选H128


In [0]:
#@title 开始转换图片

Model = "H128" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR", "Quick96"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [0]:
#@title 开始图片转视频
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练  
**注意：**workspace.zip只包含src和dst，不包含model,否则模型会被覆盖。！！！

In [0]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "Quick96" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#重新覆盖解压，提升加载速度，可节省很多时间
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  -o workspace.zip


# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# 安装Python依赖
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image

# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"
!python $cmd 

# 工具


使用脚本练模型的时候常常会需要上传，解压，压缩，下载等，下面为大家提供一个解压和压缩的脚本  
所有压缩的文件都存在zip目录，名字都叫data.zip，压缩前选一下压缩那个文件夹下的内容。  
解压的时候也是上传一个data.zip，选择解压到哪里就好了  




In [0]:
#@title 添加中....

# 其他

 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com
 
 使用教程：https://www.wxp123.me
 
 
不求别的，只求在github给个小星星^_^

https://github.com/dream80/DeepFaceLab_Colab

右上角star 谢谢!!!